<a href="https://colab.research.google.com/github/onenechan/GEE_turtrial/blob/main/DL%E6%A4%9C%E8%A8%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LandSat

In [2]:
# Install google earth engine api
!python -c "from oauth2client import crypt"
!pip install earthengine-api

# Authorization
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=-C1CX_pJAB9swvd3TpjNSy0nRR8CJc2c-fK7j497UPc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g61RgDYc4Xn3GazloS7EhLZeT5nOYSa0mF-xFDFvZOEr93XuTHBLMo

Successfully saved authorization token

In [3]:
from IPython.display import Image
import ee
import ee.mapclient

ee.Initialize()

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [6]:
import os
import pprint
import time
import urllib.error
import urllib.request
from google.colab import files

In [7]:
# Input data
Area1 = ee.Geometry.Rectangle(139.7,35.6,139.8,35.7)
Landsat8_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate('2019-06-01','2019-06-15').min()

In [8]:
def TrueColor(data,area,gain):
  """
  TrueColor画像の可視化を行うメソッド。
  image : ee.ImageCollection()で指定した衛星データ
  area  : ee.Geometryで得られた位置データ
  """
  red = data.expression("b('B4')*gain",{"gain":gain})
  green = data.expression("b('B3')*gain",{"gain":gain})
  blue = data.expression("b('B2')*gain",{"gain":gain})
  
  # TrueColorのバンドを重ねたデータを作成する。
  bands = ee.Image(red).addBands(green).addBands(blue)
  image = bands.clip(area)
  image_url = image.getThumbUrl({'min':0,'max':1,'dimensions':'800'})
  return Image(url=image_url)

In [9]:
TrueColor(Landsat8_collection,Area1,1)

In [10]:
def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file, open(dst_path, 'wb') as local_file:
            local_file.write(web_file.read())
            files.download(dst_path)
    except urllib.error.URLError as e:
        print(e)

In [20]:
def IdentifyCloud(data,area):
  """
  雲を判別して可視化するメソッド
  data : ee.ImageCollection()で指定した衛星データ
  area  : ee.Geometryで得られた位置データ
  """
  
  #雲と雪の判別を行うためのバンド6,7の反射率の基準値を設定
  cloud_lim = 0.1
  
  #()?がif,:()?がelse if,:がelse文に対応
  #b()でバンドを選択できる
  #バンド6,7がともに基準値以上であれば雲と判断し、青色にする
  red = data.expression("(b('B6')>lim && b('B7')>lim)? 0"+ ": b('B4')",{"lim":cloud_lim})
  green = data.expression("(b('B6')>lim && b('B7')>lim)? 0" + ": b('B3')",{"lim":cloud_lim})
  blue = data.expression("(b('B6')>lim && b('B7')>lim)? 1"+": b('B2')",{"lim":cloud_lim})
  
  #TrueColorのバンドを重ねたデータを作成
  image = ee.Image(red).addBands(green).addBands(blue)
  
  #データの範囲を選択
  clipped_image = image.clip(area)
  
  #画像のサムネイルの作成と画像の可視化
  image_url = clipped_image.getThumbUrl({'min':0,'max':1,'dimensions':'800'})
  print(image_url)
  return image_url #Image(url=image_url)

In [21]:
img_url = IdentifyCloud(Landsat8_collection,Area1)
download_file(img_url, 'test.png')

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a689e1cf98069e2897696872c2a8f9fa-720c2830ca7fc3aa09bedac57dba57d9:getPixels


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

夜間光

In [14]:
VIIRS_Nighttime_Day = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG").filterDate('2019-06-01','2019-06-15').min()

In [17]:
dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date('2019-06-01', '2017-06-15'));
nighttime = dataset.select('avg_rad');
nighttimeVis = {min: 0.0, max: 60.0};
Map.setCenter(-77.1056, 38.8904, 8);
Map.addLayer(nighttime, nighttimeVis, 'Nighttime');

NameError: ignored